# Gremlin Workshop - Section 01: Select & Filter Query Steps
In this section, you will start with some basic Gremlin queries that are used to find vertices or edges with given attributes.  Those attributes can be labels, unique IDs, or properties.  Feel free to change any of the queries in this (and subsequent) notebooks to see how changing the query may result in different outcomes.

Happy Traversing!

## Setup

Before you start, ensure you have run notebook _00-Setup_ to create the dataset with which we'll be working.

## Graph Model

As we've mentioned prior, we will be using a modified version of the IMDb (Internet Movie Database) dataset for this workshp.  We have converted this data into the following graph data model representing certain aspects of the data as vertices, edges, and properties.  While not part of this workshop, it is important to understand graph data modeling.  Please refer to our Graph Reference Architectures GitHub repo for more details on how to perform graph data modeling:  https://github.com/aws-samples/aws-dbs-refarch-graph

Here's the application graph data model:

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>



## Select a Vertex and its Properties

When selecting graph elements, bear the following in mind:

 - Neptune allows you to supply custom vertex and edge IDs. Use these if possible to lookup a vertex or edge by ID. See the _User Supplied IDs_ section in the [documentation](https://docs.aws.amazon.com/neptune/latest/userguide/access-graph-gremlin-differences.html).
 - If you can't use an ID to lookup an element, use the _minimum number_ of label and property predicates necessary to uniquely identify the element or the set of elements you want to find.
 - When returning results, select only the properties necessary to satisfy your query, rather than returning vertices and edges in their entirety.

### 01.01 - Find a Vertex or Edge by ID

To start, you will perform the most simple of Gremlin operations to find a Vertex (or an Edge) by it's unique ID.  Each Vertex and Edge in Neptune has a unique ID.  IDs can be random (such as UUIDs) or can also be something meaningful, if also unique.  Examples of this can be email addresses signifying a person vertex.  In the IMDb dataset, IDs are already provided for both Artists and their works (e.g. movies, TV shows, etc.).  For example, if you go to the [IMDb page for Kevin Bacon](https://www.imdb.com/name/nm0000102/) you will notice the last part of the URL is `nm0000102` is the unique identifer for Kevin.

In the following example, we use Kevin's identifer to find his vertex.

In [ ]:
%%gremlin

g.V('nm0000102')

You can do the same with an edge.  Each edge has a unique ID.  The original IMDb dataset did not have a first-class representation of the relationships.  We had to manufacture these using the unique IDs of the Artists and their productions.  In this case, we used a composite ID combining the unique IDs of the vertices on either side of a relationship to derive the unique ID for each edge.  For example, the edge representing the relationship from [Kevin Bacon](https://www.imdb.com/name/nm0000102/) to [Footloose](https://www.imdb.com/title/tt0087277/) is `tt0087277-actor-nm0000102`.  Try querying for this edge:

In [ ]:
%%gremlin

g.E('tt0087277-actor-nm0000102')

### 01.02 - Displaying Property Values

Finding a vertex or edge is great, but only showing the vertex or edge ID output isn't very interesting.  Let's start showing some more useful data by exposing the property values associated with each.  You can show property values for a given vertex or edge using the Gremlin steps for values() and valueMap():

Consult the following documentation:
 - [`valueMap()`](http://tinkerpop.apache.org/docs/current/reference/#valuemap-step)
 - [`values()`](http://tinkerpop.apache.org/docs/current/reference/#_values_step)
 - [`elementMap()`](https://tinkerpop.apache.org/docs/current/reference/#elementmap-step)
 
In the following example, let's see all properties associated with Kevin Bacon's vertex:

In [ ]:
%%gremlin

g.V('nm0000102').valueMap()

We can do the same with the movie "Footloose".

In [ ]:
%%gremlin

g.V('tt0087277').valueMap()

Note that neither the unique ID nor the label of the vertex are shown when using `valueMap()`.  If you need this information, you can use the following notation of `valueMap().with(WithOptions.tokens)` to expose these properties:

In [ ]:
%%gremlin

g.V('tt0087277').valueMap().with(WithOptions.tokens)

The output above can become cumbersome to read via a single line.  Sometimes we may want to view each of the properties in separate lines in these notebooks.  We can do this by "unfolding" the results with the `unfold()` step:

In [ ]:
%%gremlin

g.V('tt0087277').valueMap().with(WithOptions.tokens).unfold()

Besides `valueMap()`, Gremlin provides another method to look at the aspects of an edge.  Along with a unique ID, a label, and various properties, edges are also defined by their incoming and outgoing vertices.  Sometimes it is beneficial to see the neighboring vertices when looking at an edge.  We can do this using the `elementMap()` step as such:

In [ ]:
%%gremlin

g.E('tt0087277-actor-nm0000102').elementMap().unfold()

### 01.03 - Filtering by Certain Properties

Sometimes you won't know the exact ID for a specific vertex or edge.  It is more likely that you would know the value of a given property associated with that vertex or edge.  In those cases, we can use the `has()` Gremlin step to find a given vertex or edge with a given property value.

Consult the following documentation:
 - [`has()`](http://tinkerpop.apache.org/docs/current/reference/#has-step)
 
 In the below example, we want to find all vertices representing the Footloose movie.

In [ ]:
%%gremlin

g.V().has('title','Footloose').
    valueMap().with(WithOptions.tokens)

Each edge and vertex in Neptune has an associated label that denotes the type of data being represented by a set of vertices or edges. Labels tend to be of very low cardinatlity.

Over time, there have been two renditions of the movie "Footloose" and a handful of TV episodes with the same name.  If we had included other media forms as vertices (e.g. vertex labels representing tvEpisodes, tvSpecials, tvSeries, etc.) and just wanted to see the movies associated with the title "Footloose," we can use the `hasLabel()` Gremlin step to only filter on one or more labels, like the following:

Consult the following documentation:
 - [`hasLabel()`](http://tinkerpop.apache.org/docs/current/reference/#has-step)

In [ ]:
%%gremlin

g.V().hasLabel('movie').has('title','Footloose').
    valueMap().with(WithOptions.tokens)

In addition to `has()` and `hasLabel()`, there are also Gremlin steps for `hasId()`, `hasKey()`, `hasValue()`, `hasNot()` and some additional patterns.  Check out the official TinkerPop documentation for the `has()` step for more details.  http://tinkerpop.apache.org/docs/current/reference/#has-step

### 01.04 - Using Conditionals

Beyond the `has()` steps, there are a few other ways to find vertices and edges based on certain search criteria.  For example, say we wanted to find all instances of "Footloose", that were filmed before 2000.  In this case, we can use one of the conditional Gremlin steps to do this.

Gremlin has a few steps for evaluating conditionals.  Including, but not limited to:

- `gt()` - greater than
- `gte()` - greater than or equal to
- `lt()` - less than
- `lte()` - less than or equal to
- `eq()` - equal to
- `neq()` - not equal

There are also set operations:

- `within()` - test of the output is within a given set
- `without()` - opposite of `within()`

There are many [other operations for conditionals](https://kelvinlawrence.net/book/Gremlin-Graph-Guide.html#tranges).

To find all versions of "Footloose" prior to 2000 we can use the following:

In [ ]:
%%gremlin

g.V().has('title','Footloose').has('year',lt(2000)).valueMap()

If you compare this output to our earlier output showing all "Footloose" titles, this only contains the titles with a release year of earlier than 2005.

Another conditional operator is also `between()`.  We can use the `between()` step to find all versions of Footloose filmed between two different years.

Further documentation:
- `between()` - http://tinkerpop.apache.org/docs/3.4.4/reference/#a-note-on-predicates

For example, if we wanted to find all versions of Footloose filmed from 2000 to 2012:

In [ ]:
%%gremlin

g.V().has('title','Footloose').has('year',between(2000,2012)).valueMap()

### Review

In the previous notebook we covered the basics of using Gremlin to look at vertices, edges, and their subsequent properties.  We also looked at how to begin filtering on property values.  We'll begin to build on these simple queries in the following notebooks and use these patterns as starting and ending conditions for traversing our graph.


### End of Section 01 - Move onto the next notebook.